# Welcome me to my PyNotebook

In [0]:
# Human Activity Recognition using Smartphone accelerometer and gyroscope dataset

## Importing packages to get drive files loaded

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Importing packages for loading and processing dataset

In [4]:
!pip install hmmlearn
from sklearn.ensemble import RandomForestClassifier
#from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import precision_recall_fscore_support as prfs
from hmmlearn.hmm import GaussianHMM
#import graphviz as gz
import numpy as np
import pandas as pd

link = 'https://drive.google.com/open?id=1F_ev5qK5bL3BuLFoCLgZ2wMAqnH-S-u0'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('X_train.txt')  
df = pd.read_csv('X_train.txt',sep='\s+',header=None)

link = 'https://drive.google.com/open?id=1ANc9W0wki9gvd9aVWnSCOeI566tsl-MC'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('y_train.txt')
df1 = pd.read_csv('y_train.txt',sep='\s+',header=None)

link = 'https://drive.google.com/open?id=1K0WgXHsm53u7VPYZAn8-GCI8EuPRre5C'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('features.txt')
names = pd.read_csv('features.txt',sep='\s+',header=None)

link = 'https://drive.google.com/open?id=1-Y1WjlaDul5je7bzn7PY4fXXrCH8DNFp'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('X_test.txt')  
df_X = pd.read_csv('X_test.txt',sep='\s+',header=None)

link = 'https://drive.google.com/open?id=1jR0ZecuI64Apoe8mqnpLsFshJTPiSzqT'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('y_test.txt')  
df_y = pd.read_csv('y_test.txt',sep='\s+',header=None)



    100% |████████████████████████████████| 153kB 13.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/9a/a4/ee/917f0de81626b684fd2139ef5df47744c35ebeacc9e950487b
Successfully built hmmlearn
1F_ev5qK5bL3BuLFoCLgZ2wMAqnH-S-u0
1ANc9W0wki9gvd9aVWnSCOeI566tsl-MC
1K0WgXHsm53u7VPYZAn8-GCI8EuPRre5C
1-Y1WjlaDul5je7bzn7PY4fXXrCH8DNFp
1jR0ZecuI64Apoe8mqnpLsFshJTPiSzqT


In [5]:
fe = list(names.iloc[:,1])
df_X.shape

(2947, 561)

### Loading dataframes of train and test dataset

In [0]:
df.columns = names.iloc[:,1]
X = df	#rows


y = df1[0].values.tolist()
df_X.columns = names.iloc[:,1]
X1 = df_X

y1 = df_y[0].values.tolist()


## Comparing scores - RF vs LDA

In [7]:
clf = RandomForestClassifier(n_estimators = 500, oob_score = True, n_jobs = -1,
							random_state =50, max_features = None,
							min_samples_leaf = 1)
clf.fit(X1, y1)
s1 = clf.oob_score_

clf1 = lda()
clf1.fit(X1,y1)
s2 = clf1.score(X1,y1)

print('RF - ',s1,'\n','LDA - ',s2)

RF -  0.9742110620970479 
 LDA -  0.996267390566678


/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


## HMM model

In [0]:
feat = []
feat_1 = []
new_feat = []
for i,j in zip(fe,clf.feature_importances_):
  feat.append([i,j])
feat.sort(key = lambda feat:feat[1], reverse=True)
#print(feat)
#print(len(feat))
c = 0
for i in feat:
  if i[1] >= 0.0001:
    feat_1.append(i)
    new_feat.append(i[0])
    c += 1

#print(feat_1)

In [9]:
X = df_X[new_feat]
clf_new = GaussianHMM(n_components = 1)
clf_new.fit(X,lengths=None)
a = (clf_new.predict(X))

print('HMM Stage: ',clf_new.n_components)
print('Features: ',X.shape[1])
print(cm(y1,clf_new.predict(X)))


HMM Stage:  1
Features:  414
[[  0   0   0   0   0   0   0]
 [496   0   0   0   0   0   0]
 [471   0   0   0   0   0   0]
 [420   0   0   0   0   0   0]
 [491   0   0   0   0   0   0]
 [532   0   0   0   0   0   0]
 [537   0   0   0   0   0   0]]


In [10]:
print('RF Features: ',X1.shape[1])
print(cm(y1,clf.predict(df_X)))

clf.fit(X,y1)
print('RF Features: ',X.shape[1])
print(cm(y1,clf.predict(X)))

RF Features:  561
[[496   0   0   0   0   0]
 [  0 471   0   0   0   0]
 [  0   0 420   0   0   0]
 [  0   0   0 491   0   0]
 [  0   0   0   0 532   0]
 [  0   0   0   0   0 537]]
RF Features:  414
[[496   0   0   0   0   0]
 [  0 471   0   0   0   0]
 [  0   0 420   0   0   0]
 [  0   0   0 491   0   0]
 [  0   0   0   0 532   0]
 [  0   0   0   0   0 537]]


In [11]:
clf_new = GaussianHMM(n_components = 2)
clf_new.fit(X,lengths=None)
a = (clf_new.predict(X))

print('HMM Stage: ',clf_new.n_components)
print('Features: ',X.shape[1])
print(cm(y1,clf_new.predict(X)))

HMM Stage:  2
Features:  414
[[  0   0   0   0   0   0   0]
 [  0 496   0   0   0   0   0]
 [  0 471   0   0   0   0   0]
 [  0 420   0   0   0   0   0]
 [470  21   0   0   0   0   0]
 [484  48   0   0   0   0   0]
 [515  22   0   0   0   0   0]]
